In [10]:
# 本代码由可视化策略环境自动生成 2019年5月11日 22:02
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m5_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df = input_1.read_df()
    tmp = m4.data.read_df().groupby('date').apply(lambda x:list(x.sort_values('market_cap_0').instrument[:20])) 
    data_1 = DataSource.write_pickle(tmp)
    return Outputs(data_1=data_1)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m5_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：每日数据处理函数，每天执行一次
def m1_handle_data_bigquant_run(context, data):
    if context.trading_day_index % context.hold_days !=0:
        return 
    # 当前的日期
    date = data.current_dt.strftime('%Y-%m-%d')
    
    # 根据日期获取调仓需要买入的股票的列表
    cur_data =  context.weight_data[date]
    stock_to_buy = list(cur_data.keys())
  
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]
    # 继续持有的股票：调仓时，如果买入的股票已经存在于目前的持仓里，那么应继续持有
    no_need_to_sell = [i for i in stock_hold_now if i in stock_to_buy]
    # 需要卖出的股票
    stock_to_sell = [i for i in stock_hold_now if i not in no_need_to_sell]
  
    # 卖出
    for stock in stock_to_sell:
        # 如果该股票停牌，则没法成交。因此需要用can_trade方法检查下该股票的状态
        # 如果返回真值，则可以正常下单，否则会出错
        # 因为stock是字符串格式，我们用symbol方法将其转化成平台可以接受的形式：Equity格式

        if data.can_trade(context.symbol(stock)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，
            #   即卖出全部股票，可参考回测文档
            
            context.order_target_percent(context.symbol(stock), 0)
    
    # 如果当天没有买入的股票，就返回
    if len(stock_to_buy) == 0:
        return

    
    # 买入
    for stock in stock_to_buy:
        if data.can_trade(context.symbol(stock)):
            # 下单使得某只股票的持仓权重达到weight，因为
            # weight大于0,因此是等权重买入
            weight = cur_data[stock]
            context.order_target_percent(context.symbol(stock), weight)
# 回测引擎：准备数据，只执行一次
def m1_prepare_bigquant_run(context):
    pass

# 回测引擎：初始化函数，只执行一次
def m1_initialize_bigquant_run(context):
    print('>>>>>>>: portfolio optimizer')
    # 加载预测数据
    context.weight_data = context.options['data'].read_pickle()
    

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.hold_days = 22
    
# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m1_before_trading_start_bigquant_run(context, data):
    pass

# 回测引擎：每日数据处理函数，每天执行一次
def m7_handle_data_bigquant_run(context, data):
    if context.trading_day_index % context.hold_days !=0:
        return 
    # 当前的日期
    date = data.current_dt.strftime('%Y-%m-%d')
    
    # 根据日期获取调仓需要买入的股票的列表
    stock_to_buy = list(context.buy_list.ix[date])
    
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]
    # 继续持有的股票：调仓时，如果买入的股票已经存在于目前的持仓里，那么应继续持有
    no_need_to_sell = [i for i in stock_hold_now if i in stock_to_buy]
    # 需要卖出的股票
    stock_to_sell = [i for i in stock_hold_now if i not in no_need_to_sell]
  
    # 卖出
    for stock in stock_to_sell:
        # 如果该股票停牌，则没法成交。因此需要用can_trade方法检查下该股票的状态
        # 如果返回真值，则可以正常下单，否则会出错
        # 因为stock是字符串格式，我们用symbol方法将其转化成平台可以接受的形式：Equity格式

        if data.can_trade(context.symbol(stock)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，
            #   即卖出全部股票，可参考回测文档
            context.order_target_percent(context.symbol(stock), 0)
    
    # 如果当天没有买入的股票，就返回
    if len(stock_to_buy) == 0:
        return

    # 等权重买入 
    weight =  1 / len(stock_to_buy)
    
    # 买入
    for stock in stock_to_buy:
        if data.can_trade(context.symbol(stock)):
            # 下单使得某只股票的持仓权重达到weight，因为
            # weight大于0,因此是等权重买入
            context.order_target_percent(context.symbol(stock), weight)
# 回测引擎：准备数据，只执行一次
def m7_prepare_bigquant_run(context):
    pass

# 回测引擎：初始化函数，只执行一次
def m7_initialize_bigquant_run(context):
    print('>>>>>>>: equal weight')
    # 加载预测数据
    context.buy_list = context.options['data'].read_pickle()
   
    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.hold_days = 22
    
# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m7_before_trading_start_bigquant_run(context, data):
    pass

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m8_run_bigquant_run(input_1, input_2, input_3):
    
    nv1 = m1.raw_perf.read_df()['algorithm_period_return']
    nv2 =m7.raw_perf.read_df()['algorithm_period_return']

    df = pd.DataFrame({'optimizer weight':nv1,'equal weight':nv2})
    T.plot(df , title='对比图')
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m8_post_run_bigquant_run(outputs):
    return outputs


m2 = M.input_features.v1(
    features="""market_cap_0
"""
)

m3 = M.instruments.v2(
    start_date='2016-01-01',
    end_date='2017-01-01',
    market='CN_STOCK_A',
    instrument_list=''
)

m4 = M.general_feature_extractor.v7(
    instruments=m3.data,
    features=m2.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m5 = M.cached.v3(
    input_1=m4.data,
    run=m5_run_bigquant_run,
    post_run=m5_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m9 = M.classical_portfolio_optimizer.v6(
    input_1=m5.data_1,
    symbols=['000002.SZA', '000333.SZA', '000009.SZA'],
    date='2018-01-01',
    weight_sum=1,
    upper_weight=0.3,
    lower_weight=0.01,
    before_start_days=252,
    target='风险平价',
    return_equal_weight_if_fail=True
)

m1 = M.trade.v4(
    instruments=m3.data,
    options_data=m9.data_1,
    start_date='',
    end_date='',
    handle_data=m1_handle_data_bigquant_run,
    prepare=m1_prepare_bigquant_run,
    initialize=m1_initialize_bigquant_run,
    before_trading_start=m1_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)

m7 = M.trade.v4(
    instruments=m3.data,
    options_data=m5.data_1,
    start_date='',
    end_date='',
    handle_data=m7_handle_data_bigquant_run,
    prepare=m7_prepare_bigquant_run,
    initialize=m7_initialize_bigquant_run,
    before_trading_start=m7_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)

m8 = M.cached.v3(
    input_1=m7.raw_perf,
    input_2=m1.raw_perf,
    run=m8_run_bigquant_run,
    post_run=m8_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports='',
    m_cached=False
)


[2019-04-30 12:30:12.978750] INFO: bigquant: input_features.v1 开始运行..

[2019-04-30 12:30:13.020846] INFO: bigquant: 命中缓存

[2019-04-30 12:30:13.022589] INFO: bigquant: input_features.v1 运行完成[0.043842s].

[2019-04-30 12:30:13.025259] INFO: bigquant: instruments.v2 开始运行..

[2019-04-30 12:30:13.050709] INFO: bigquant: 命中缓存

...

[2019-04-30 12:31:24.001134] INFO: bigquant: backtest.v8 开始运行..

[2019-04-30 12:31:24.027041] INFO: bigquant: 命中缓存

[2019-04-30 12:31:25.005444] INFO: bigquant: backtest.v8 运行完成[1.004295s].

[2019-04-30 12:31:25.009376] INFO: bigquant: cached.v3 开始运行..

[2019-04-30 12:31:25.278021] INFO: bigquant: cached.v3 运行完成[0.268633s].